In [1]:
!pip install -U transformers[torch]
!pip install -U huggingface_hub
!pip install evaluate
!pip install bert_score
!pip install git+https://github.com/google-research/bleurt.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.5/417.5 kB 10.8 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.24.6
    Uninstalling huggingface-hub-0.24.6:
      Successfully uninstalled huggingface-hub-0.24.6
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 22.3 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages tha

In [2]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=e6492c1acc2a0a02833a782b6e0b96bdcd6527cce3a8d2263f7422444b285b67
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [3]:
import pandas as pd
import numpy as np
import requests
from io import BytesIO
import os

In [4]:
train_url = 'https://raw.githubusercontent.com/viswa3024/aiml-capstone-project-email/main/question-answering/Dataset-1/aiml-qa-train.xlsx'
response = requests.get(train_url)
if response.status_code == 200:
    tain_data = BytesIO(response.content)
    try:
        df = pd.read_excel(tain_data)
    except Exception as e:
        print("Error reading the Excel file:", e)
else:
    print("Failed to fetch the file. Status code:", response.status_code)


In [5]:
df.head()

,question,answer
0,What is the difference between concatenation v...,Concatenation combines two tensors by adding t...
1,What is the difference between concatenation v...,Concatenation is often used to combine differe...
2,Why are derivatives substracted from weights?,The derivative of the loss function at a point...
3,Why are derivatives substracted from weights?,The intuition behind adjusting the weights by ...
4,Describe a process/pipeline for generating rep...,Select a pretrained model suitable for the tas...


In [6]:
df.shape

(1316, 2)

In [7]:
dev_url = 'https://raw.githubusercontent.com/viswa3024/aiml-capstone-project-email/main/question-answering/Dataset-1/aiml-qa-dev.xlsx'
response = requests.get(dev_url)
if response.status_code == 200:
    dev_data = BytesIO(response.content)
    try:
        df_val = pd.read_excel(dev_data)
    except Exception as e:
        print("Error reading the Excel file:", e)
else:
    print("Failed to fetch the file. Status code:", response.status_code)


In [8]:
df_val.head()

,question,answer1,answer2
0,Does the maximum value of 'n' in the n-gram mo...,The maximum value of n in an n-gram model is n...,The maximum value of 'n' in an n-gram model do...
1,How CNN works?,Convolutional Neural Networks (CNNs) use convo...,CNN is a type of neural network that is common...
2,How is NMT trained? Is it common to use pairs ...,Neural Machine Translation (NMT) is typically ...,"Yes, training NMT models involves using parall..."
3,What is the process of learning POS tags?,The process of learning POS tags involves trai...,POS tags are learned by training machine learn...
4,how to handle multi lingual situations in NLP?,Handling multilingual situations in NLP involv...,It involves techniques such as language identi...


In [9]:
df_val.shape

(80, 3)

In [10]:
test_url = 'https://raw.githubusercontent.com/viswa3024/aiml-capstone-project-email/main/question-answering/Dataset-1/aiml-qa-test.xlsx'
response = requests.get(test_url)
if response.status_code == 200:
    test_data = BytesIO(response.content)
    try:
        df_test = pd.read_excel(test_data)
    except Exception as e:
        print("Error reading the Excel file:", e)
else:
    print("Failed to fetch the file. Status code:", response.status_code)


In [11]:
df_test.head()

,question,answer1,answer2
0,How we can effectively convert 2D images to 1D?,Converting images to 1D data may not be effect...,"To effectively convert 2D images to 1D, use te..."
1,Can we utilize an autoencoder to perform dimen...,"Yes, autoencoders can be applied to numerical ...","Yes, autoencoders can be used for dimensionali..."
2,What is NLP's current biggest challenge that i...,The main challenges of NLP is finding and coll...,NLP models struggle with tasks that require re...
3,Which problems cannot be solved by Neural netw...,While neural networks have shown great success...,"Neural networks are powerful, but they may str..."
4,Is scaling necessary for SVM?,"Yes, scaling the input data is generally recom...",Scaling the input data is advisable when utili...


In [12]:
df_test.shape

(120, 3)

In [13]:
import pandas as pd
import numpy as np
import requests
from io import BytesIO
import os
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, TrainingArguments, Trainer, GPT2LMHeadModel, DataCollatorForLanguageModeling
from transformers import set_seed
import evaluate

In [14]:
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")
meteor = evaluate.load("meteor")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [15]:
torch.manual_seed(42)
set_seed(42)

In [16]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', bos_token='<|startoftext|>',
                                          eos_token='<|endoftext|>', pad_token='<|pad|>', sep_token='<|sep|>')
model = GPT2LMHeadModel.from_pretrained('gpt2').cuda()
model.resize_token_embeddings(len(tokenizer))

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Embedding(50260, 768)

In [17]:
datacollator = DataCollatorForLanguageModeling(tokenizer = tokenizer, mlm=False)

In [18]:
class QADataset(Dataset):
    def __init__(self, data, tokenizer):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        for index, row in data.iterrows():
            encodings_dict = tokenizer('<|startoftext|>' + row["question"] + '<|sep|>' + row["answer"] + '<|endoftext|>', truncation=True, max_length=300, padding="max_length", return_tensors='pt')
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx]

In [19]:
class ValQADataset(Dataset):
    def __init__(self, data, tokenizer):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        for index, row in data.iterrows():
          encodings_dict = tokenizer('<|startoftext|>' + row["question"] + '<|sep|>', truncation=True, max_length=120, padding="max_length", return_tensors='pt')
          if((encodings_dict['input_ids'][0][119] != torch.Tensor(np.array([50259]))) and (encodings_dict['input_ids'][0][119] != torch.Tensor(np.array([50258])))):
            encodings_dict['input_ids'][0] = torch.cat((encodings_dict['input_ids'][0][np.r_[:119]],torch.Tensor(np.array([50258]))),0)
          self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
          self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx]

In [20]:
class TestQADataset(Dataset):
    def __init__(self, data, tokenizer):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        for index, row in data.iterrows():
          encodings_dict = tokenizer('<|startoftext|>' + row["question"] + '<|sep|>', truncation=True, max_length=120, return_tensors='pt')
          self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
          self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx]

In [21]:
train_dataset = QADataset(df, tokenizer)
val_dataset = ValQADataset(df_val,tokenizer)

<ipython-input-18-4833a31b834d>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
<ipython-input-18-4833a31b834d>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))
<ipython-input-19-c06aa83ace77>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
<ipython-input-19-c06aa83ace77>:11: UserWarning: To copy construct from a tensor, it is recommended

In [22]:
# Define compute_metrics function
def compute_metrics(eval_pred, eval_dataset, df):
    decoded_preds = []
    references = [df['answer1'], df['answer2']]
    refs = []

    # Generate predictions for the validation dataset
    for i, sample_input in enumerate(eval_dataset):
        temp_input = sample_input[0][sample_input[0] != torch.Tensor(np.array([50259]))]
        temp_input = temp_input[None, :]
        metric_outputs = model.generate(temp_input.cuda(), min_new_tokens=60, max_new_tokens=200, penalty_alpha=0.6, top_k=4, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
        decoded_preds.append(tokenizer.decode(metric_outputs[0]))

    final_preds = []
    for j in range(len(decoded_preds)):
        lst = decoded_preds[j].split('<|sep|>')
        if len(lst) >= 2:
            temp_decoded = lst[1].replace("<|endoftext|>", "")
            final_preds.append(temp_decoded.split('___')[0])
        else:
            print(f'{j}: {lst[0]}')

        temp_refs = []
        for k in range(len(references)):
            temp_refs.append(references[k][j])
        refs.append(temp_refs)

    # Compute BLEU, ROUGE, and METEOR metrics
    results_bleu = bleu.compute(predictions=final_preds, references=refs)
    results_rouge = rouge.compute(predictions=final_preds, references=refs)
    results_meteor = meteor.compute(predictions=final_preds, references=refs)

    # Return the computed metrics
    return {
        'bleu': results_bleu['bleu'],
        'rouge1': results_rouge['rouge1'],
        'rouge2': results_rouge['rouge2'],
        'rougeL': results_rouge['rougeL'],
        'meteor': results_meteor['meteor']
    }

In [23]:
# Define TrainingArguments
training_args = TrainingArguments(
    output_dir="./output",
    overwrite_output_dir=True,
    num_train_epochs=20,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    eval_steps=400,
    save_steps=800,
    warmup_steps=500,
    load_best_model_at_end=True,
    eval_strategy="epoch",
    save_strategy="epoch",
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    metric_for_best_model="rougeL",  # Change this to "rougeL"
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    fp16=True,
    report_to='none',
    save_total_limit=1
)

In [24]:
def preprocess_logits_for_metrics(logits, labels):
    """
    Original Trainer may have a memory leak.
    This is a workaround to avoid storing too many tensors that are not needed.
    """
    pred_ids = torch.argmax(logits, dim=-1)
    return pred_ids, labels

In [25]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=datacollator,
    compute_metrics = lambda pred: compute_metrics(pred, val_dataset, df_val),
    preprocess_logits_for_metrics = preprocess_logits_for_metrics)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [26]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Epoch,Training Loss,Validation Loss,Bleu,Rouge1,Rouge2,Rougel,Meteor
0,63.654300,61.829449,0.000000,0.023963,0.005556,0.019663,0.010376
1,7.208600,6.721503,0.069947,0.274090,0.119019,0.230216,0.185203
2,3.890900,4.332821,0.035974,0.201063,0.067012,0.182559,0.166823
4,3.122800,3.373741,0.054679,0.248189,0.092451,0.217865,0.278176
5,2.929100,3.232644,0.064792,0.272786,0.111306,0.237890,0.288277
6,2.740200,3.089228,0.073088,0.295270,0.122970,0.253343,0.294701
8,2.494400,2.877471,0.124628,0.366298,0.167609,0.311007,0.316653
9,2.361400,2.806943,0.123488,0.343633,0.158984,0.293881,0.312817
10,2.214500,2.761451,0.128211,0.358025,0.169136,0.301714,0.322454
12,2.020700,2.622946,0.124027,0.336759,0.154262,0.283350,0.301406


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # typ

TrainOutput(global_step=820, training_loss=6.809984243206862, metrics={'train_runtime': 4210.318, 'train_samples_per_second': 6.251, 'train_steps_per_second': 0.195, 'total_flos': 4005729331200000.0, 'train_loss': 6.809984243206862, 'epoch': 19.87878787878788})

In [ ]:
trainer.save_model()

In [27]:
test_dataset = TestQADataset(df_test,tokenizer)

<ipython-input-20-6db5a894922d>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
<ipython-input-20-6db5a894922d>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))


In [28]:
temp_output = []
for i, sample_input in enumerate(test_dataset):
    #print("i: ", i)
    if(len(sample_input[0]) == 119 and sample_input[0][119] != torch.Tensor(np.array([50258]))):
        sample_input[0] = torch.cat((sample_input[0][np.r_[:119]],torch.Tensor(np.array([50258]))),0)
    sample_output = model.generate(sample_input.cuda(), min_new_tokens = 60, max_new_tokens = 200, penalty_alpha=0.6, top_k=4, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
    temp_output.append(tokenizer.decode(sample_output[0]))
final_output =[]
for j in range(len(temp_output)):
    lst = temp_output[j].split('<|sep|>')
    if (len(lst) >= 2):
        temp_decoded = lst[1].replace("<|endoftext|>","")
        final_output.append(temp_decoded.split('___')[0])
    else:
        final_output.append("")

In [44]:
df_test["Generated"] = final_output
df_test.to_csv('GeneratedAnswergpt2_small_dataset_1_v1.csv')

In [30]:
pred = df_test['Generated']
ref = [df_test['answer1'], df_test['answer2']]

In [34]:
# Update score_evaluate function to use BLEU, ROUGE, and METEOR
def score_evaluate(predictions, references):
    preds = []
    refs = []

    # Prepare the predictions and references
    for i in range(len(predictions)):
        #print("i:", i)
        preds.append(predictions[i])
        temp_refs = []
        for j in range(len(references)):
            temp_refs.append(references[j][i])
        refs.append(temp_refs)

    # Calculate BLEU, ROUGE, and METEOR scores
    results_bleu = bleu.compute(predictions=preds, references=refs)
    results_rouge = rouge.compute(predictions=preds, references=refs)
    results_meteor = meteor.compute(predictions=preds, references=refs)

    # Print evaluation metrics
    print("BLEU Score : " + str(results_bleu['bleu']))
    print("ROUGE-1 Score : " + str(results_rouge['rouge1']))
    print("ROUGE-2 Score : " + str(results_rouge['rouge2']))
    print("ROUGE-L Score : " + str(results_rouge['rougeL']))
    print("METEOR Score : " + str(results_meteor['meteor']))

In [ ]:
# Evaluate the model predictions
score_evaluate(pred, ref)

BLEU Score : 0.13623116655312617
ROUGE-1 Score : 0.37436314967837725
ROUGE-2 Score : 0.17491522444404123
ROUGE-L Score : 0.31025103620103023
METEOR Score : 0.33369815484321613


In [32]:
df_generated = pd.read_csv("/content/GeneratedAnswergpt2_small_dataset_1_v1.csv")

In [33]:
df_generated.head(10)

,Unnamed: 0,question,answer1,answer2,Generated
0,0,How we can effectively convert 2D images to 1D?,Converting images to 1D data may not be effect...,"To effectively convert 2D images to 1D, use te...","To convert 2D images to 1D, we can use the con..."
1,1,Can we utilize an autoencoder to perform dimen...,"Yes, autoencoders can be applied to numerical ...","Yes, autoencoders can be used for dimensionali...","Yes, an autoencoder can be used to perform dim..."
2,2,What is NLP's current biggest challenge that i...,The main challenges of NLP is finding and coll...,NLP models struggle with tasks that require re...,NLP is a powerful and challenging
3,3,Which problems cannot be solved by Neural netw...,While neural networks have shown great success...,"Neural networks are powerful, but they may str...","No, Neural networks are not suitable for solvi..."
4,4,Is scaling necessary for SVM?,"Yes, scaling the input data is generally recom...",Scaling the input data is advisable when utili...,"Yes, scaling is necessary for SVM. It is not n..."
5,5,Adding layer to a machine learning model can l...,Increasing the number of layers may lead to ov...,The architecture of the model depends on the d...,Adding layer to a machine learning model can l...
6,6,is comparision of model predictions important?,Ensuring the model's right prediction in real ...,"Yes, comparing model outputs against ground tr...",It is important to consider the impact of mode...
7,7,What is the way to invert the association rules?,"Yes, by inverting association rules, we can id...","To invert association rules, identify items wi...",Invert the association rules by applying the s...
8,8,What is the good number of filters we can have?,The number of output channels in a CNN layer c...,The number of filters in a CNN layer depends o...,The number of filters we can have depends on t...
9,9,How are Part-of-Speech (POS) tags learned in n...,POS tags are learned using supervised or unsup...,POS tags can be learned through supervised lea...,POS tags are learned by the speaker during the...


In [35]:
model.save_pretrained("qna_gpt2_small_Dataset_1_v1")
tokenizer.save_pretrained("qna_gpt2_small_Dataset_1_v1")

('qna_gpt2_small_Dataset_1_v1/tokenizer_config.json',
 'qna_gpt2_small_Dataset_1_v1/special_tokens_map.json',
 'qna_gpt2_small_Dataset_1_v1/vocab.json',
 'qna_gpt2_small_Dataset_1_v1/merges.txt',
 'qna_gpt2_small_Dataset_1_v1/added_tokens.json')

In [36]:
from huggingface_hub import notebook_login

In [37]:
notebook_login()

In [38]:
repo_name="qna_gpt2_small_Dataset_1_v1"

model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/kkasiviswanath/qna_gpt2_small_Dataset_1_v1/commit/a7bfce4309ff197ef5ae5fbe9b7a3d4ba261c97d', commit_message='Upload tokenizer', commit_description='', oid='a7bfce4309ff197ef5ae5fbe9b7a3d4ba261c97d', pr_url=None, pr_revision=None, pr_num=None)

In [39]:
# Define the cleaning function
def clean_answer(response):
    lst = response.split('<|sep|>')
    if len(lst) >= 2:
        response = lst[1].replace("<|endoftext|>", "").replace("<|pad|>", "").replace("<|startoftext|>", "")
        response = response.split('___')[0]
    return response

# Define the function to generate an answer based on a question
def generate_answer(question: str):
    prompt = f"<|startoftext|> {question} <|sep|>"

    # Generate the answer using the model
    sample_outputs = model.generate(
        tokenizer(prompt, return_tensors='pt')['input_ids'].cuda(),  # Moving input to GPU if necessary
        min_new_tokens=60,
        max_new_tokens=200,
        penalty_alpha=0.6,
        top_k=4,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id
    )

    # Clean and decode the answer
    answer = clean_answer(tokenizer.decode(sample_outputs[0]))
    return answer

In [40]:
# Test the generate_answer function with a sample question
sample_question = "Can we utilize an autoencoder to perform dimensionality reduction on numerical datasets?"
generated_answer = generate_answer(sample_question)

# Print the generated answer
print("Question: ", sample_question)
print("Generated Answer: ", generated_answer)

Question:  Can we utilize an autoencoder to perform dimensionality reduction on numerical datasets?
Generated Answer:  Yes, an autoencoder can be used to perform dimensionality reduction on numerical datasets.                                         


In [41]:
from transformers import pipeline

# Load the pipeline with the custom GPT-2 model
qa_pipeline = pipeline('text-generation', model='kkasiviswanath/qna_gpt2_small_Dataset_1_v1')

# Define a function to generate answers using the pipeline
def generate_answer_with_pipeline(question: str):
    prompt = f"<|startoftext|> {question} <|sep|>"
    output = qa_pipeline(prompt, max_new_tokens=200, top_k=4, pad_token_id=50256, return_full_text=False)
    answer = clean_answer(output[0]['generated_text'])
    return answer


config.json:   0%|          | 0.00/907 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.07k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/999k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/71.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [43]:
# Example usage
question = "Can we utilize an autoencoder to perform dimensionality reduction on numerical datasets?"
answer = generate_answer_with_pipeline(question)
print("Generated Answer:", answer)

Generated Answer: An autoencodER can perform dimensionality reduction on numerical datasets by performing dimensionality reduction on the dimensionality of the input data.
